<a href="https://colab.research.google.com/github/hamhead1005/CS-4800-Class-Project/blob/main/Google_Speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade google-cloud-speech

In [ ]:
!pip install ez_setup

In [ ]:
!pip install moviepy

In [ ]:
!pip install imageio-ffmpeg

In [ ]:
pip install pysoundfile

In [ ]:
pip install --upgrade google-cloud-storage

In [ ]:
pip install --upgrade protobuf

In [ ]:
#upload file from PC to local dir. (if needed)
from google.colab import files
uploaded = files.upload()

In [ ]:
# Block (1)

#vars
transcriptFileCount = 0 #Used to give transcript files unique names when uploaded to GCS

In [ ]:
#imports
import io
import os
import IPython
import json
import re


# Setup for download from GCS (google cloud storage)
from google.colab import auth
from google.cloud import storage
from google.protobuf.json_format import MessageToJson
auth.authenticate_user()

project_id = '[cs4800-project-333101]'
!gcloud config set project {'cs4800-project-333101'}

# Download the file from given Google Cloud Storage bucket.
#!gsutil cp gs://cs4800_bucket/interview_sample.mp4 /content/trans.mp4 #(Short MP4 File for testing)
#!gsutil cp gs://cs4800_bucket/Long_Recording.mp3 /content/Long_Recording.mp3 (Long MP3 for testing)
!gsutil cp gs://cs4800_bucket/cs4800-project-333101-d0f5d3441758.json /content/cs4800-project-333101-d0f5d3441758.json #Speech to Text/ Cloud Credential

!gsutil cp gs://cs4800_bucket/Responses/demo_response1.mp4 /content/demo_response1.mp4 #For DEMO
!gsutil cp gs://cs4800_bucket/Responses/demo_response2.mp4 /content/demo_response2.mp4 #For DEMO
!gsutil cp gs://cs4800_bucket/Responses/demo_response3.mp4 /content/demo_response3.mp4 #For DEMO
!gsutil cp gs://cs4800_bucket/Responses/demo_response4.mp4 /content/demo_response4.mp4 #For DEMO

Updated property [core/project].
Copying gs://cs4800_bucket/cs4800-project-333101-d0f5d3441758.json...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      
Copying gs://cs4800_bucket/Responses/demo_response1.mp4...
\ [1 files][ 55.8 MiB/ 55.8 MiB]                                                
Operation completed over 1 objects/55.8 MiB.                                     
Copying gs://cs4800_bucket/Responses/demo_response2.mp4...
/ [1 files][  1.1 MiB/  1.1 MiB]                                                
Operation completed over 1 objects/1.1 MiB.                                      
Copying gs://cs4800_bucket/Responses/demo_response3.mp4...
- [1 files][ 10.6 MiB/ 10.6 MiB]                                                
Operation completed over 1 objects/10.6 MiB.                                     
Copying gs://cs4800_bucket/Responses/demo_response4.mp4...
| [1 file

In [ ]:
#Create credential object and test access
from google.cloud import speech

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/cs4800-project-333101-d0f5d3441758.json'

client = speech.SpeechClient()

# Change the File_tag Below to change which reponse video we are converting
- The number corresponds to the number in the demo_reponse#.mp4 file

In [ ]:
file_tag = 4 #Used to change target local file

#Convert MP4 to MP3
import moviepy
from moviepy.editor import *

mp4_file = '/content/demo_response'+ str(file_tag) +'.mp4'
mp3_file = '/content/audio_file.mp3'

vc = moviepy.editor.VideoFileClip(mp4_file)
ac = vc.audio

ac.write_audiofile(mp3_file)

ac.close()
vc.close()

[MoviePy] Writing audio in /content/audio_file.mp3


100%|██████████| 1535/1535 [00:02<00:00, 653.81it/s]

[MoviePy] Done.


In [ ]:
#audio file too be transcribed (Short)
media_file_name_mp3 = '/content/audio_file.mp3'

#load media file for API
with open(media_file_name_mp3, 'rb') as f1:
    byte_data_mp3 = f1.read()
audio_mp3 = speech.RecognitionAudio(content=byte_data_mp3)

#Play audio
IPython.display.Audio(media_file_name_mp3) #used for testing

In [ ]:
#Upload MP3 to Cloud 
#Save Results to Google Cloud Storage
storage_client = storage.Client.from_service_account_json('cs4800-project-333101-d0f5d3441758.json')

filename = 'Converted_mp3s/audio_file.mp3' # Save location + Name of File (includes counter)
bucket = storage_client.get_bucket('cs4800_bucket')                       # Name of Bucket
blob = bucket.blob(filename)                                              # Save Location in Bucket + filename
blob.upload_from_filename('audio_file.mp3')                              # Local File to be saved

In [ ]:
#Long File Transcription
media_uri = 'gs://cs4800_bucket/Converted_mp3s/audio_file.mp3' #Target File from Google Cloud Service
long_aud_mp3 = speech.RecognitionAudio(uri=media_uri)

config_mp3_enhanced = speech.RecognitionConfig(
  sample_rate_hertz=48000,
  enable_automatic_punctuation=True,
  language_code='en-US',
  use_enhanced= True,
  model='video' #Added Precaution
)

#Translate Operation
operation = client.long_running_recognize(
    config=config_mp3_enhanced,
    audio = long_aud_mp3
)

#Text Response
response_long_mp3 = operation.result(timeout=90)
print(response_long_mp3)

results {
  alternatives {
    transcript: "Describe the last project you worked on including any obstacles and your contributions to its success."
    confidence: 0.8569173812866211
  }
}
results {
  alternatives {
    transcript: " Hmm, previous employer task me with creating an internal online learning and training program for employees. The purpose of the program was to make sure all employees receive proper training on certain topics including customer service compliance with legal requirements and workplace ethics. I began by researching other similar Training Systems to figure out what worked and what didn\'t next. I use Java to"
    confidence: 0.8464963436126709
  }
}
results {
  alternatives {
    transcript: " Simple program, which I then used to upload training courses."
    confidence: 0.7913317084312439
  }
}
results {
  alternatives {
    transcript: " After testing the simplified program."
    confidence: 0.8746377825737
  }
}
results {
  alternatives {
    transcript: 

# Confidence Grading Function

(Not)1-10(confident) Score
- Well, hmm, umm, err
  - 10 - count
- Count Pauses
  - 10 - count
- Length of Reply
  - 10 - count

Avg: 3 options --> Final Confidence Score

In [ ]:
# Count Confidence Variables
FillerWords_score = 10
paues_score = 11 #First Response does not count as a pause
word_score = 0
word_list = 0
wordCount = 1

for result in response_long_mp3.results:
    paues_score -= 1 #Count Breaks
    str_trans = str(result.alternatives[0].transcript) #Get Transcript as a String
    word_list = (re.findall(r'\w+', str(str_trans))) #Convert String to List
    wordCount += len(word_list) #Count Number of Words
    for i in range(len(word_list)):
      if(word_list[i].lower() == 'hmm' or word_list[i].lower() == 'well' or word_list[i].lower() == 'umm' or word_list[i].lower() == 'err'):
        if(FillerWords_score != 0):
          FillerWords_score -= 1 #Count Filler words


print("Fillers Words Score: " + str(FillerWords_score))
print("Pauses Score: " + str(paues_score))
print("Words Used: " + str(wordCount))


#Word Length Score:
if(wordCount >= 150):
  word_score = 10
elif (wordCount >= 120 and wordCount <= 149):
  word_score = 9
elif (wordCount >= 100 and wordCount <= 119):
  word_score = 8
elif (wordCount >= 90 and wordCount <= 99):
  word_score = 7
elif (wordCount >= 70 and wordCount <= 89):
  word_score = 6
elif (wordCount >= 50 and wordCount <= 69):
  word_score = 5
elif (wordCount >= 40 and wordCount <= 49):
  word_score = 4
elif (wordCount >= 30 and wordCount <= 39):
  word_score = 3
elif (wordCount >= 20 and wordCount <= 29):
  word_score = 2
elif (wordCount >= 10 and wordCount <= 19):
  word_score = 1
elif (wordCount <= 9):
  word_score = 0

print("Word Score: " + str(word_score))

# Calculate Confidence Score
confidence_score = (FillerWords_score + word_score + paues_score)/3 
print("Confidence Score: " + str(confidence_score))

Fillers Words Score: 8
Pauses Score: 6
Words Used: 143
Word Score: 9
Confidence Score: 7.666666666666667


In [ ]:
#Save output as JSON locally
rows = []

for res in response_long_mp3.results:
    for alt in res.alternatives:
        rows.append({"transcript": alt.transcript})

rows.append({"Confidence Score": confidence_score})
with open("transcript.json" , "w+") as file:
    json.dump(rows, file, indent=2)

In [ ]:
#Save Results to Google Cloud Storage
storage_client = storage.Client.from_service_account_json('cs4800-project-333101-d0f5d3441758.json')

filename = 'transcripts/transcript_' + str(transcriptFileCount) + '.json' # Save location + Name of File (includes counter)
bucket = storage_client.get_bucket('cs4800_bucket')                       # Name of Bucket
blob = bucket.blob(filename)                                              # Save Location in Bucket + filename
blob.upload_from_filename('transcript.json')                              # Local File to be saved

transcriptFileCount += 1
print(transcriptFileCount) #Prints the number of transcript file

3
